In [1]:
# Import Libraries
import cv2
import numpy as np
import time
from tkinter import *
from PIL import Image, ImageTk


In [2]:
# Global Variables
cap = None
qr_detector = cv2.QRCodeDetector()
running = False

video_label = None
result_label = None
time_label = None
last_decoded_text = ""


In [3]:
# Access Video Feed with Permission Check
def access_video():
    global cap
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 960)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 540)

    if not cap.isOpened():
        print("Requesting Video Access Permission...")
        cap.release()
        cap = None
        return False
    return True


In [4]:
# Frame Processing
def process_frame():
    global running
    if not running or cap is None:
        return

    ret, frame = cap.read()
    if not ret:
        print("Failed to read video feed")
        return

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Start timing for detect + decode
    start_time = time.time()
    decoded_text, points, _ = qr_detector.detectAndDecode(gray)
    end_time = time.time()
    total_time = end_time - start_time  # in seconds

    if points is not None:
        points = points[0].astype(int)
        for i in range(len(points)):
            pt1 = tuple(points[i])
            pt2 = tuple(points[(i + 1) % len(points)])
            cv2.line(frame, pt1, pt2, (255, 0, 0), 2)  # Blue bounding box

        # Draw center point
        center_x = int(np.mean(points[:, 0]))
        center_y = int(np.mean(points[:, 1]))
        cv2.circle(frame, (center_x, center_y), 4, (0, 0, 255), -1)

    global last_decoded_text

    if decoded_text:
        result_label.config(text=f"Decoded: {decoded_text}")
        if decoded_text != last_decoded_text:
            time_label.config(text=f"Detection + Decoding Time: {total_time * 1000:.2f} ms")
            last_decoded_text = decoded_text
    else:
        result_label.config(text="No QR Code Detected")
        last_decoded_text = ""
        time_label.config(text="Detection + Decoding Time: -")

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(rgb_frame)
    imgtk = ImageTk.PhotoImage(image=img)
    video_label.imgtk = imgtk
    video_label.config(image=imgtk)

    video_label.after(10, process_frame)


In [5]:
# GUI Creation
def create_gui(root):
    global video_label, result_label, time_label

    video_frame = Frame(root)
    video_frame.pack(pady=10)

    video_label = Label(video_frame)
    video_label.pack()

    result_label = Label(root, text="", font=("Arial", 14))
    result_label.pack(pady=10)

    time_label = Label(root, text="", font=("Arial", 12))
    time_label.pack()

    button_frame = Frame(root)
    button_frame.pack(pady=10)

    start_button = Button(button_frame, text="Start Scan", command=start_scan, bg="lightgreen", font=("Arial", 12), width=15)
    start_button.pack(side=LEFT, padx=5)

    stop_button = Button(button_frame, text="Stop Scan", command=stop_scan, bg="lightcoral", font=("Arial", 12), width=15)
    stop_button.pack(side=LEFT, padx=5)


In [6]:
# Start & Stop Functions
def start_scan():
    global running
    if access_video():
        running = True
        process_frame()
    else:
        result_label.config(text="Unable to access video feed.")

def stop_scan():
    global running, cap
    running = False
    if cap:
        cap.release()
        cap = None
    result_label.config(text="Scan stopped.")
    video_label.config(image="")


In [7]:
# Launch GUI
root = Tk()
root.title("QR Code Scanner")
root.geometry("1000x685")
create_gui(root)
root.mainloop()
